<a href="https://colab.research.google.com/github/lamtung1997/Solving_PDEs_Deep_learning/blob/main/1_Poisson.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# SETUP
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import time
import math
import itertools
pi = math.pi

np.random.seed(1)
tf.random.set_seed(1)

In [2]:
# MODEL
nn = 32
inputs = keras.Input(shape=(2,), name='points')
hidden_1 = layers.Dense(nn, activation='tanh', name='hidden_1')(inputs)
outputs = layers.Dense(1, activation='linear', name="u")(hidden_1)
model = keras.Model(inputs=inputs, outputs=outputs, name='poisson')
model.summary()

Model: "poisson"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
points (InputLayer)          [(None, 2)]               0         
_________________________________________________________________
hidden_1 (Dense)             (None, 32)                96        
_________________________________________________________________
u (Dense)                    (None, 1)                 33        
Total params: 129
Trainable params: 129
Non-trainable params: 0
_________________________________________________________________


In [3]:
# DATA
import itertools

n = 40                      # number of points in a border
c = np.linspace(0, 1, n)    # coordinates from 0 to 1 with uniform distribution
P_in, P_border = [],[]
for i, j in itertools.product(range(n), range(n)):
    p = [c[i], c[j]]
    if(i == 0 or i == n-1 or j == 0 or j == n-1):
        P_border.append(p)
    else:
        P_in.append(p)

# upsampling P_border that its size is equal to P_in
for i in range(len(P_in)-len(P_border)):
    j = i%(len(P_border))
    P_border.append(P_border[j])

# set batch_size and shuffle_size
batch_size = 4
bs = int(len(P_in))

# convert numpy array, P_in and P_border to Tensorflow Dataset
P_in = tf.convert_to_tensor(P_in, dtype=tf.float32)
P_border = tf.convert_to_tensor(P_border, dtype=tf.float32)

P_in = tf.data.Dataset.from_tensor_slices((P_in))
P_in = P_in.shuffle(buffer_size=bs).batch(batch_size).prefetch(2).cache()

P_border = tf.data.Dataset.from_tensor_slices((P_border))
P_border = P_border.shuffle(buffer_size=bs).batch(batch_size).prefetch(2).cache()

In [4]:
print("Number of data: ", int(P_in.cardinality()*batch_size*2))

Number of data:  2888


In [5]:
# CHOOSE OPTIMIZER
optimizer = tf.keras.optimizers.Adam()

In [6]:
# TRAIN_STEP FUNCTION
@tf.function
def train_step(P_in, P_border):
    loss_in, loss_border = [0], [0]
    with tf.GradientTape() as tape:
        # interior loss
        for i in range(P_in.shape[0]):
            z = tf.reshape(P_in[i], shape=(1,2))
            with tf.GradientTape(persistent=True) as tape1:
                tape1.watch(z)
                with tf.GradientTape(persistent=True) as tape2:  
                    tape2.watch(z)
                    u = model(z)[0][0]
                # u_x and u)y
                u_x = tape2.gradient(u, z)[0][0]
                u_y = tape2.gradient(u, z)[0][1]
            # u_xx and u_yy
            u_xx = tape1.gradient(u_x, z)[0][0]
            u_yy = tape1.gradient(u_y, z)[0][1]

            f = -2*pi*pi* tf.math.sin(pi*z[0][0]) * tf.math.sin(pi*z[0][1])
            loss_in += tf.math.square(u_xx + u_yy - f)

        # border loss
        for j in range(P_border.shape[0]):
            z = tf.reshape(P_border[j], shape=(1,2))
            u = model(z)[0][0]
            loss_border += tf.math.square(u)

        # loss
        batch_loss = loss_in + loss_border

    # update w
    grads = tape.gradient(batch_loss, model.weights)
    optimizer.apply_gradients(zip(grads, model.weights))

    return batch_loss

In [7]:
# TRAIN FUNCTION
losses = []
def train(learning_rate):
    optimizer.learning_rate.assign(learning_rate)
    loss = 0
    for (P_in_batch,P_border_batch) in itertools.zip_longest(P_in,P_border):
        batch_loss = train_step(P_in_batch, P_border_batch)
        loss += batch_loss
    losses.append(loss.numpy())

In [8]:
# TRAIN
i = 0
max_ite = 2000
lr = 1e-3
start = time.time()
print("Learning rate:", lr)
while(True):
    if(i > max_ite):
        break
    start_ite = time.time()
    train(lr)
    if(i%1 == 0):
        print("[%4s] loss = %12.2f \t %4.2fs \t less than %3.2f minutes to finish" % (i, losses[i], time.time() - start_ite, (time.time() - start_ite)*(max_ite - i)/60))
    if(i > 150):
        if((losses[i] / np.min(losses[0: i-100])) > 0.5):
            lr = lr/10
            if(lr < 1e-4):
                break
            print("\nNew learning rate:", lr)
    i = i + 1


print("Total time: %.2f minutes" % ((time.time() - start)/60))

Learning rate: 0.001
[   0] loss =    116244.70 	 2.55s 	 less than 84.97 minutes to finish
[   1] loss =     34265.00 	 0.38s 	 less than 12.79 minutes to finish
[   2] loss =     24024.88 	 0.39s 	 less than 13.04 minutes to finish
[   3] loss =     18255.84 	 0.40s 	 less than 13.36 minutes to finish
[   4] loss =     13143.57 	 0.42s 	 less than 14.09 minutes to finish
[   5] loss =      9155.13 	 0.39s 	 less than 13.02 minutes to finish
[   6] loss =      6371.42 	 0.39s 	 less than 13.08 minutes to finish
[   7] loss =      4664.54 	 0.39s 	 less than 13.08 minutes to finish
[   8] loss =      3747.82 	 0.39s 	 less than 12.98 minutes to finish
[   9] loss =      3286.81 	 0.38s 	 less than 12.74 minutes to finish
[  10] loss =      3042.02 	 0.39s 	 less than 13.02 minutes to finish
[  11] loss =      2889.44 	 0.39s 	 less than 12.85 minutes to finish
[  12] loss =      2775.31 	 0.39s 	 less than 12.82 minutes to finish
[  13] loss =      2677.61 	 0.39s 	 less than 12.92 min

In [9]:
def u(network,x,y):
    z = tf.convert_to_tensor([[x, y]])
    u = float(network(z))
    return u

In [10]:
model.save("/content/drive/My Drive/Colab Notebooks/Keras/Solving PDEs/1. Poisson")

INFO:tensorflow:Assets written to: /content/drive/My Drive/Colab Notebooks/Keras/Solving PDEs/1. Poisson/assets


In [11]:
model1 = keras.models.load_model("/content/drive/My Drive/Colab Notebooks/Keras/Solving PDEs/1. Poisson")

In [12]:
n = 10                      # number of points in a border
c = np.linspace(0, 1, n)    # coordinates from 0 to 1 with uniform distribution
P = []
for i, j in itertools.product(range(n), range(n)):
    p = [c[i], c[j]]
    P.append(p)

def exact_sol(x,y):
    return math.sin(pi*x)*math.sin(pi*y)

err = 0
for i in range(len(P)):
    err += (u(model1, P[i][0], P[i][1]) - exact_sol(P[i][0], P[i][1]))**2

L2 = math.sqrt(err/len(P))
print("L2 = ", L2)

L2 =  0.004604502143762609
